In [49]:
import pims
import trackpy as tp # http://soft-matter.github.io/trackpy/v0.5.0/tutorial/walkthrough.html
import os.path as path
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
import pandas as pd


fps = 30 # frames per second
dt = 1/fps # time interval between frames
microns_per_pixel = 0.1805  # scale of the microscope

# use pims package when processing video, to not run out of RAM
@pims.pipeline
def example_function(frame):
    """
    do something to each frame
    """
    x_min = 245
    x_max = 515
    y_min = 80
    y_max = 350

    frame[y_min:y_max,x_min:x_max] = 0 # set to 0 pixel values in a rectangle
    
    return frame


#@pims.pipeline
#def select_bright_centers(image):
    # tresh = ski.filters.threshold_otsu(image) # automatically select treshold by Otsu method; might not work
   # tresh = 170 # or manually select it
   # image = image > tresh # only look at pixels that are brighter than the threshold
   # image = ski.morphology.flood_fill(image,(0,0),False) # remove the bright edge of the picture
   # image = ski.morphology.remove_small_objects(image,min_size=30) # remove small objects below critical size
    #return image

@pims.pipeline
def select_bright_centers(image):
    tresh = 130
    image=image<tresh
    image = ski.morphology.remove_small_objects(image,min_size=140)
    image=ski.morphology.isotropic_closing(image,3)
    image = ski.morphology.flood_fill(image,(0,0),False)
    image = ski.morphology.flood_fill(image,(0,0),True)
    image=~image
    return image
    

fname = "1_2024_03_11_13_14_26.avi"
datadir = "C:/Users/sisni/Downloads"
rawframes = pims.as_grey( pims.PyAVVideoReader( path.join(datadir, fname)) )
frames = select_bright_centers(rawframes)

# check visually if select_bright_centers looks ok
plt.imshow(rawframes[1], cmap="gray")
plt.show()
plt.imshow(frames[1], cmap="gray")
plt.show()






## first find particle positions
nframes = len(frames)
    
features_in_python = []
for num, img in enumerate(frames[:nframes]):
    if num%100 == 0:
        print("progress = ", num/len(frames)*100, " %")
        
    label_image, number_of_labels = ski.measure.label(img, return_num=True)
    N_particles = 0
    for region in ski.measure.regionprops(label_image):
        # Everywhere, skip small and large areas
        # if region.area > 120 or region.area < 50:
        #     number_of_labels -= 1
        #     # print("skipping area = ", region.area)
        #     continue
        # print("accepting area = ", region.area)
        
        N_particles += 1
        features_in_python.append(
            {'y': region.centroid[0],
                   'x': region.centroid[1],
                   'frame': num,}
            )

    print("Number of found particles = ", N_particles)

    
features = pd.DataFrame(data=features_in_python) # object that contains the particle positions at each frame






## second, connect the particle positions into trajectories


search_range = 21 # how much a particle can move between frames, I think
trajec = tp.link_df(features, search_range, memory=1)


col_names = ['dx', 'dy', 'x', 'y', 'frame', 'particle']
# Creating an empty dataframe to store results
data = pd.DataFrame(np.zeros(shape=(1, 6), dtype=np.int64), columns=col_names)
for item in set(trajec.particle):
    sub = trajec[trajec.particle==item]
    if sub.shape[0]<=100:
        # Cases in which particle only has <100 rows of data
        pass
    else:    
        print('Deriving velocities for particle:', str(item))
        dvx = pd.DataFrame(np.gradient(sub.x), columns=['dx',])
        dvy = pd.DataFrame(np.gradient(sub.y), columns=['dy',])
    
        new_df = pd.concat((dvx, dvy, sub.x.reset_index(drop=True), sub.y.reset_index(drop=True),
                            sub.frame.reset_index(drop=True), sub.particle.reset_index(drop=True)),
                            axis=1, names=col_names, sort=False)
        data = pd.concat((data, new_df), axis=0)
    
# This is to get rid of the first 'np.zeros' row and to reset indexes
data = data.reset_index(drop=True)
data = data.drop(0)
data = data.reset_index(drop=True)

# save all in trajec variable
trajec = data

# plot  all trajectories
number_of_trajectories_minus_one = np.max(np.array(trajec["particle"]))
for i in range(number_of_trajectories_minus_one):
    
    # select coordinates for particle i
    xs = np.array(trajec[ trajec["particle"] == i ]["x"]) # have to convert to numpy arrays 
    ys = np.array(trajec[ trajec["particle"] == i ]["y"])
#     
    
    plt.plot(xs,ys)
    
plt.show()


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [59]:
with open('resultados_trajec_v2.txt', 'w') as f:
    # Escribir encabezados de las columnas
    f.write('{:<15}\t{:<15}\t{:<15}\t{:<15}\t{:<15}\t{:<15}\n'.format('x', 'y', 'frame', 'particle', 'r', 'theta'))
    
    # Escribir cada fila con los valores alineados
    for index, row in trajec.iterrows():
        x = '{:.6f}'.format(row['x'])  # Formatear el valor de x,y,... con 6 decimales
        y = '{:.6f}'.format(row['y'])  
        frame = row['frame'] 
        particle = row['particle']  
        r = '{:.6f}'.format(row['r'])  
        theta = '{:.6f}'.format(row['theta'])
        # Escribir la fila en el archivo
        f.write('{:<15}\t{:<15}\t{:<15}\t{:<15}\t{:<15}\t{:<15}\n'.format(x, y, frame, particle, r, theta))